In [93]:
import pandas as pd
from datetime import timedelta, datetime


In [94]:
def addcycles(dataset):
    # Create the cycles list, incrementing total cycles everytime a large jump up is made
    cycles = [0]
    counter = 0
    
    listcolumns = list(dataset.columns.values)
    temp = [x for x in listcolumns if "LV Input" in x]
    
    a = dataset[temp[0]]
    b = dataset['Status']
    highestcount = 0
    for i in range(1, len(a)):
        if b[i] == 'failed':
            counter = 0
            cycles.append(highestcounter)
            continue
        if a[i]>30 and a[i-1]<4:
            counter +=1
        cycles.append(counter)
        highestcounter = counter
    
    
    #Create the cycles_until_failue list
    previousfail = 0
    cycles_until_failure = []
    
    for i in range(1,len(a)):
        if b[i]=='failed' and b[i-1] == 'working':
            for j in range(i-previousfail):
                cycles_until_failure.append(cycles[i-1]-cycles[j+previousfail])
            cycles_until_failure.append(0)    
        elif b[i]=='failed':
            cycles_until_failure.append(0)
        elif b[i]=='working' and b[i-1] == 'failed':
            previousfail = i
        else:
            continue
    while len(cycles_until_failure) < len(a):
        cycles_until_failure.append(None)
    
    # Add Both Cycles columns to the end of the data
    dataset['Cycles_survived'] = cycles
    dataset['Cycles_until_failure'] = cycles_until_failure

In [95]:
def operationtime(dataset):
    
    # Setup the dataset and extra lists
    times = dataset['Time']
    df = pd.DataFrame({'Time':times})
    time1 = pd.to_datetime(df['Time'])
    threshhold = datetime.strptime("2021/01/01 10:46:00", "%Y/%m/%d %H:%M:%S") - datetime.strptime("2021/01/01 10:44:00", "%Y/%m/%d %H:%M:%S")
    status = dataset['Status']
    operationtime = [time1[1]-time1[0]]
    
    #Add operationtime
    temp = time1[0]-time1[0]
    mostrecentworking = time1[0]
    for i in range(1,len(time1)):
        if status[i] == 'working':
            if status[i-1] == 'failed':
                temp = time1[0] - time1[0]
            delta = time1[i]-time1[i-1]
            if(delta<threshhold):
                temp = temp + delta
            operationtime.append(temp)
            mostrecentworking = temp
        else:
            operationtime.append(mostrecentworking)
    dataset["Operation_time"] = operationtime
    
    #Add Time until failure
    failuretime = []
    a = dataset['Time']
    b = dataset['Status']
    previousfail = 0
    time_until_failure = []
    for i in range(1, len(b)):
        if b[i]=='failed' and b[i-1] == 'working':
            for j in range(i-previousfail):
                time_until_failure.append(datetime.strptime(a[i-1],"%Y-%m-%d %H:%M:%S.%f")-datetime.strptime(a[j+previousfail],"%Y-%m-%d %H:%M:%S.%f"))
            time_until_failure.append(time1[0]-time1[0])    
        elif b[i]=='failed':
            time_until_failure.append(time1[0]-time1[0])
        elif b[i]=='working' and b[i-1] == 'failed':
            previousfail = i
        else:
            continue
    while len(time_until_failure) < len(a):
        time_until_failure.append(None)
    dataset["Time_until_failure"] = time_until_failure

In [96]:
def removeextremetemps(f):
    listcolumns = list(f.columns.values)
    temp = [x for x in listcolumns if "(C)" in x]
    tempdf = f
    for i in range(5):
        tempdf = tempdf.loc[tempdf[temp[i]] > 0]
        tempdf = tempdf.loc[tempdf[temp[i]] < 2000]
    return tempdf

In [102]:
def deltavoltage(f):
    listcolumns = list(f.columns.values)
    LVinput = [x for x in listcolumns if "LV Input" in x]
    HVinput = [x for x in listcolumns if "HV Input" in x]
    LVoutput = [x for x in listcolumns if "LV Out" in x]
    HVoutput = [x for x in listcolumns if "HV Out" in x]
    LVin = f[LVinput[0]]
    HVin = f[HVinput[0]]
    LVout = f[LVoutput[0]]
    HVout = f[HVoutput[0]]
#     print(HVout[0:3])
    deltaHV = []
    deltaLV = []
    for i in range(len(LVin)):
        deltaHV.append(HVout[i]-HVin[i])
        deltaLV.append(LVout[i]-LVin[i])
    f["delta_HV"] = deltaHV
    f["delta_LV"] = deltaLV

In [103]:
def addaveragetemp(f):
    
    listcolumns = list(f.columns.values)
    temps = [x for x in listcolumns if "(C)" in x]
    alltemps = []
    averagetemps = []
    for i in range(len(temps)):
        alltemps.append(f[temps[i]])
    for i in range(len(alltemps[0])):
        tot = 0
        for j in range(5):
            tot += alltemps[j][i]
        averagetemps.append(tot/5)
    f["averagetemps"] = averagetemps

In [106]:
#Old Stuff

a = pd.read_csv("C:\\remove_extra_rows\\heater_5_jan-march.csv")
# addcycles(a)
# # operationtime(a)
# a = removeextremetemps(a)
deltavoltage(a)
a.head()
# addaveragetemp(a)
# a.to_csv("C:\\remove_extra_rows\\heater_5.csv", index = False)

0   -10.263997
1   -10.416936
2   -10.483912
Name: 906 <B HV Out> (VAC), dtype: float64


,Unnamed: 0,Time,109 <B LV Input> (VAC),110 <B HV Input> (VAC),906 <B HV Out> (VAC),907 <B LV Out> (VAC),306 <Htr5 sn side 10 in> (C),307 <Htr5 sn side 5 in> (C),308 <Htr5 center> (C),309 <Htr5 5 in> (C),310 <Htr5 10 in> (C),114 <HTR5 Leakage> (MA),909 <Htr5 Power> (WAT),113 <HTR5 Current> (AAC),Sequence Number,Status,delta_HV,delta_LV
0,0,2022-01-31 14:17:04.446,0.805920,-0.686555,-10.263997,0.801085,23.304,23.298,23.282,23.293,23.248,0.031911,-0.001230,-0.001535,sn5001,working,-9.577442,-0.004836
1,1,2022-01-31 14:17:08.431,0.800812,-0.696785,-10.416936,0.796007,23.316,23.282,23.279,23.271,23.261,0.032551,-0.001242,-0.001560,sn5001,working,-9.720151,-0.004805
2,2,2022-01-31 14:17:12.431,0.798256,-0.701265,-10.483912,0.793466,23.307,23.270,23.288,23.286,23.261,0.032870,-0.001218,-0.001535,sn5001,working,-9.782647,-0.004790
3,3,2022-01-31 14:17:16.431,0.796978,-0.699985,-10.464776,0.792196,23.310,23.273,23.269,23.286,23.261,0.032999,-0.001175,-0.001484,sn5001,working,-9.764791,-0.004782
4,4,2022-01-31 14:17:20.431,0.795065,-0.698705,-10.445640,0.790295,23.313,23.263,23.288,23.293,23.251,0.032934,-0.001193,-0.001509,sn5001,working,-9.746935,-0.004770


In [110]:
for i in range(1,7):
    if i ==4:
        continue
    name = "C:\\remove_extra_rows\\heater_" + str(i) + "_jan-march.csv"
    a = pd.read_csv(name)
    addcycles(a)
    operationtime(a)
    a = removeextremetemps(a)
    a.reset_index(drop = True, inplace = True)
    addaveragetemp(a)
    outputname = "C:\\allcolumnsadded\\heater_" + str(i) + ".csv"
    a.to_csv(outputname, index = False)

0    6551.988099
1    6554.317776
2    6552.503632
Name: 901 <A HV Out> (VAC), dtype: float64
0    6554.317776
1    6552.503632
2    6549.276512
Name: 901 <A HV Out> (VAC), dtype: float64
0    6551.988099
1    6554.317776
2    6552.503632
Name: 901 <A HV Out> (VAC), dtype: float64
0   -10.263997
1   -10.416936
2   -10.483912
Name: 906 <B HV Out> (VAC), dtype: float64
0   -9.738131
1   -9.728563
2   -9.757267
Name: 906 <B HV Out> (VAC), dtype: float64


In [8]:
# oldstuff




# If want to test on small section of dataset
# ds = pd.read_csv("C:\labeled-data\heater_1_jan-march.csv")
# new_ds = ds.iloc[-300000:]
# new_ds.to_csv('C:\labeled-data\smallheater_1.csv', index_label = None)
# ds1 = pd.read_csv("C:\labeled-data\smallheater_1.csv")

# Full heater 1 dataset
ds1 = pd.read_csv("C:\labeled-data\heater_1_jan-march.csv")
ds2 = pd.read_csv("C:\labeled-data\heater_2_jan-march.csv")
ds3 = pd.read_csv("C:\labeled-data\heater_3_jan-march.csv")
ds4 = pd.read_csv("C:\labeled-data\heater_4_jan-march.csv")
ds5 = pd.read_csv("C:\labeled-data\heater_5_jan-march.csv")
ds6 = pd.read_csv("C:\labeled-data\heater_6_jan-march.csv")

addcycles(ds1)
addcycles(ds2)
addcycles(ds3)
addcycles(ds4)
addcycles(ds5)
addcycles(ds6)

ds1 = operationtime(ds1)
ds2 = operationtime(ds2)
ds3 = operationtime(ds3)
ds4 = operationtime(ds4)
ds5 = operationtime(ds5)
ds6 = operationtime(ds6)

ds1.to_csv("C:\Almost done\heater_1_jan-marchmod.csv", index = False)
ds2.to_csv("C:\Almost done\heater_2_jan-marchmod.csv", index = False)
ds3.to_csv("C:\Almost done\heater_3_jan-marchmod.csv", index = False)
ds4.to_csv("C:\Almost done\heater_4_jan-marchmod.csv", index = False)
ds5.to_csv("C:\Almost done\heater_5_jan-marchmod.csv", index = False)
ds6.to_csv("C:\Almost done\heater_6_jan-marchmod.csv", index = False)

In [62]:
# Old stuff
f = pd.read_csv("C:\\remove_extra_rows\\heater_5_jan-march.csv")
print(f.shape)
f = removeextremetemps(f)
print(f.shape)

(581931, 16)
(369440, 16)


In [69]:
f = pd.read_csv("C:\\remove_extra_rows\\heater_5_jan-march.csv")
print(f.shape)
result_ds = f.loc[f['203 <Htr3 sn side 10 in> (C)'] > 0]
result_ds1 = result_ds.loc[f[' <Htr5 sn side 5 in> (C)'] > 0]
result_ds2 = result_ds1.loc[f['308 <Htr5 center> (C)'] > 0]
result_ds3 = result_ds2.loc[f['310 <Htr5 10 in> (C)'] > 0]
result_ds4 = result_ds3.loc[f['309 <Htr5 5 in> (C)'] > 0]
print(result_ds4.shape)

result_ds4.to_csv("C:\\remove_extra_rows\\heater_5_jan-marchpurged.csv",index=False)

(581931, 16)


KeyError: '203 <Htr3 sn side 10 in> (C)'